In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv(
    "../../../Datasets/RegExpPURE/PURE_train.csv",
    usecols=["Requirement", "Req/Not Req"],
)

test_df = pd.read_csv(
    "../../../Datasets/RegExpPURE/PURE_test.csv",
    usecols=["Requirement", "Req/Not Req"],
)

valid_df = pd.read_csv(
    "../../../Datasets/RegExpPURE/PURE_valid.csv",
    usecols=["Requirement", "Req/Not Req"],
)

In [3]:
train_X = train_df["Requirement"].values
train_y = train_df["Req/Not Req"].map({"Req": 1, "Not_Req": 0}).values

test_X = test_df["Requirement"].values
test_y = test_df["Req/Not Req"].map({"Req": 1, "Not_Req": 0}).values

valid_X = valid_df["Requirement"].values
valid_y = valid_df["Req/Not Req"].map({"Req": 1, "Not_Req": 0}).values

## Running DistilBERT with Hugging Face Transformers

In [4]:
from transformers import DistilBertTokenizerFast, DistilBertModel

d:\SWE Class\Github Desktop\Classification-of-Software-Requirements\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
distilbert_model = DistilBertModel.from_pretrained("../../../Models/DistilBERT_Pretrained")


# Create the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [6]:
# Save the model in the Model folder
distilbert_model.save_pretrained("../../../Models/DistilBERT_Pretrained")

In [7]:
import torch

In [8]:
# Tokenize the train_X data
train_encoded = tokenizer(
    train_X.tolist(), truncation=True, padding=True, max_length=64, return_tensors="pt"
)

# Convert the tokenized data to tensors
input_ids = torch.tensor(train_encoded["input_ids"])
attention_mask = torch.tensor(train_encoded["attention_mask"])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13644\1178858753.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(train_encoded["input_ids"])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_13644\1178858753.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(train_encoded["attention_mask"])


In [9]:
outputs = distilbert_model(input_ids=input_ids, attention_mask=attention_mask)
outputs.last_hidden_state.shape

: 